In [3]:
# === 1. Importar bibliotecas ===
import re
import pandas as pd
import os

# === 2. Configurações e Caminhos ===
# ATENÇÃO: Mude a extensão de input para .xlsx
input_path = "pr_review_comments_classificar_por_tipo.xlsx"
output_comments_path = "pr_review_comments_classificados_contextual.csv"
output_pr_dominance_path = "pr_dominancia_qp2a.csv"

# === 3. Carregar dados ===
column_mapping = {
    'id_review_comment': 'comment_id',
    'id_PR': 'pr_id',
    'body_review_comment': 'comment_body',
    'Titulo_PR': 'pr_title',
    'body_PR': 'pr_body'
}
# === MUDANÇA AQUI: usando pd.read_excel() ===
# Assume que os dados estão na primeira aba (sheet_name=0)
try:
    df = pd.read_excel(input_path, sheet_name=0)
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em {input_path}. Verifique o caminho e o nome do arquivo.")
    exit()

df = df.rename(columns=column_mapping)
# Converte a coluna pr_id para inteiro, se necessário (importante para o agrupamento)
df['pr_id'] = df['pr_id'].astype(int)


# --- 4. Regras Heurísticas e Classificação (QP2) ---
# Define os padrões Regex e seus respectivos rótulos
classification_rules = {
    # Nível de Criticidade
    'MANDATORIO_BLOQUEIO': {
        'Natureza_Critica': 'MANDATORIO_BLOQUEIO',
        'Tipo_Comentario': 'NA_CRITICA',
        'Patterns': [r"must", r"have to", r"required", r"needs to be", r"should not", r"won'?t merge", r"blocker", r"missing", r"fix this", r"can'?t accept", r"please\s+reconsider", r"it's broken"]
    },
    'SUGESTIVO_OPCIONAL': {
        'Natureza_Critica': 'SUGESTIVO_OPCIONAL',
        'Tipo_Comentario': 'NA_CRITICA',
        'Patterns': [r"could\s+be", r"consider(ing)?", r"maybe", r"suggestion", r"optional", r"if you want", r"recommend", r"perhaps", r"might be"]
    },

    # Tipos de Comentário (Modos de Falha)
    'BUG_LOGICA': {
        'Natureza_Critica': 'NA_CRITICA',
        'Tipo_Comentario': 'BUG_LOGICA',
        'Patterns': [r"bug", r"fix", r"error", r"defect", r"wrong", r"issue", r"incorrect", r"unexpected", r"doesn'?t work", r"should\s+be"]
    },
    'DESIGN_ARQUITETURA': {
        'Natureza_Critica': 'NA_CRITICA',
        'Tipo_Comentario': 'DESIGN_ARQUITETURA',
        'Patterns': [r"design", r"architecture", r"refactor", r"pattern", r"api", r"coupling", r"duplicate", r"responsibility"]
    },
    'TESTE_DOC': {
        'Natureza_Critica': 'NA_CRITICA',
        'Tipo_Comentario': 'TESTE_DOC',
        'Patterns': [r"test case", r"unit test", r"add( a)? test", r"coverage", r"document(ation)?", r"comment", r"explain", r"verify"]
    },
    'ESTILO_FORMATO': {
        'Natureza_Critica': 'NA_CRITICA',
        'Tipo_Comentario': 'ESTILO_FORMATO',
        'Patterns': [r"\b(blank|extra)\s*(line|space)", r"indent", r"format(ting)?", r"style guide", r"convention", r"naming"]
    },

    # Neutro / Interação
    'NA_APROVACAO_INTERACAO': {
        'Natureza_Critica': 'NA_APROVACAO_INTERACAO',
        'Tipo_Comentario': 'NA_APROVACAO_INTERACAO',
        'Patterns': [r"\blgtm\b", r"looks good", r"approved", r"merge (it|this)", r"thank(s| you)", r"appreciate", r"good job", r"nice", r"congrats", r"\+1", r"assignment", r"ping", r"question"]
    }
}

# Mapeamento de Prioridade para Modos de Falha (Tipo_Comentario)
priority_map = {
    'MANDATORIO_BLOQUEIO': 5,
    'SUGESTIVO_OPCIONAL': 4,
    'BUG_LOGICA': 3,
    'DESIGN_ARQUITETURA': 3,
    'TESTE_DOC': 2,
    'ESTILO_FORMATO': 2,
    'NA_APROVACAO_INTERACAO': 0,
    'UNCLASSIFIED': -1
}


# --- 5. Função de Classificação Única CONTEXTUALIZADA ---
def classify_contextual(row):
    # 1. Concatena os três campos para análise contextual
    # Converte para string para tratar valores NaN ou nulos
    text_to_analyze = str(row['comment_body']) + " " + str(row['pr_title']) + " " + str(row['pr_body'])
    text_to_analyze = text_to_analyze.lower()

    # Inicialização
    natureza_critica_label = "UNCLASSIFIED"
    tipo_comentario_label = "UNCLASSIFIED"
    current_priority = -1

    for label, info in classification_rules.items():
        for p in info['Patterns']:
            if re.search(p, text_to_analyze):

                # Regra de Dominância da Natureza Crítica
                if 'MANDATORIO' in info['Natureza_Critica']:
                    natureza_critica_label = info['Natureza_Critica']

                # Regra de Dominância do Tipo de Comentário (Modo de Falha)
                if priority_map.get(label, 0) > current_priority:
                     tipo_comentario_label = info['Tipo_Comentario']
                     current_priority = priority_map.get(label, 0)

                # Se a maior prioridade ainda for neutra, ajusta para o neutro mais alto
                if current_priority <= 0:
                    natureza_critica_label = 'NA_APROVACAO_INTERACAO'
                    tipo_comentario_label = 'NA_APROVACAO_INTERACAO'

                break # Sai dos padrões e vai para a próxima label

    # Se nada foi encontrado (priority = -1), mantém o default UNCLASSIFIED
    if current_priority == -1:
        natureza_critica_label = "UNCLASSIFIED"
        tipo_comentario_label = "UNCLASSIFIED"

    return pd.Series([natureza_critica_label, tipo_comentario_label])

# --- 6. Aplicar e Criar Novas Colunas ---
print("Iniciando Classificação Contextual...")
df[['Natureza_Critica', 'Tipo_Comentario']] = df.apply(classify_contextual, axis=1)

print("✅ Classificação por Comentário Concluída.")

# --- 7. Executar a Agregação por Dominância (PR Única) para QP2a ---

# Mapeamento de Rank para Dominância (3=Mandatório, 2=Sugestivo, 1=Neutro)
criticality_rank_map = {
    'MANDATORIO_BLOQUEIO': 3,
    'SUGESTIVO_OPCIONAL': 2,
    'NA_APROVACAO_INTERACAO': 1,
    'UNCLASSIFIED': 1,
}

# Mapeamento Reverso
reverse_label_map = {
    3: 'MANDATORIO',
    2: 'SUGESTIVO',
    1: 'NEUTRO',
}

# 1. Criar Rank Numérico no DF de Comentários
df['criticality_rank'] = df['Natureza_Critica'].map(criticality_rank_map)

# 2. Agrupar pela PR (pr_id) e encontrar o Rank MÁXIMO (Dominante)
df_pr_dominance = df.groupby('pr_id')['criticality_rank'].max().reset_index()

# 3. Criar o Rótulo Final da PR (Natureza_Critica_Dominante)
df_pr_dominance['Natureza_Critica_Dominante'] = df_pr_dominance['criticality_rank'].map(reverse_label_map)

# --- 8. Salvar Resultados ---

# 8.1. Salvar o arquivo de comentários classificados (para QP2b)
# Mantido em CSV para facilidade de leitura posterior em Jamovi/outros
df.to_csv(output_comments_path, index=False, encoding="utf-8")
print(f"✅ Comentários classificados (nível comentário) salvos em: {output_comments_path}")

# 8.2. Salvar o arquivo de Dominância por PR (para QP2a)
df_pr_dominance[['pr_id', 'Natureza_Critica_Dominante']].to_csv(output_pr_dominance_path, index=False, encoding="utf-8")
print(f"✅ Dominância de Criticidade (nível PR) salva em: {output_pr_dominance_path}")

print("\n--- Próximo Passo: Rodar Qui-Quadrado (QP2a) ---")

Iniciando Classificação Contextual...
✅ Classificação por Comentário Concluída.
✅ Comentários classificados (nível comentário) salvos em: pr_review_comments_classificados_contextual.csv
✅ Dominância de Criticidade (nível PR) salva em: pr_dominancia_qp2a.csv

--- Próximo Passo: Rodar Qui-Quadrado (QP2a) ---
